# LLMs Transfer Learning

Notebook by: Samson Bakos

Based on the documentation available at: https://huggingface.co

## Example 1: Text Classification

We're going to do this using HF's default objects for simplicity, but you can also do it in Tensorflow/Keras, or PyTorch

#### Load the [Yelp Reviews Dataset](https://huggingface.co/datasets/yelp_review_full)

In [1]:
from datasets import load_dataset

# load dataset 
dataset = load_dataset("yelp_review_full")

# print example from training set
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

Each example in this dataset includes a text review from Yelp, along with a star rating (1-5, mapped to labels 0-4)

Our task is predict the star rating given the review (5 class classification)

#### Preprocess Text

In [2]:
from transformers import AutoTokenizer

# use the default preprocessor 
# important to ensure expected input to our model (i.e. same lemmatization modelling, stopwords, etc)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
   # Map function 
    # padding and truncation control for variable length sequences
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# apply to all datasets with .map(). Built in function of the HF datasets class
tokenized_datasets = dataset.map(tokenize_function, batched=True)

This is slow as heck (unless this is already stored in memory) and that was just preprocessing and loading. 

Lets take a reduced subset of the data to speed up for demo purposes

In [3]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

#### Train

Load DistilBert itself

This will throw a warning, but its fine.

Its basically just telling us this model isn't trained on a specific task yet

In [4]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=5)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We have some setup to do before we start, but there are pretty useful premade functions for us here.

There are ALOT of [hyperparameters for training](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments). 

There are some more important settings like number of epochs, optimizer function (i.e. ADAM, SGD), learning rate, loss function, etc. Reasonable setting for settings are built in as defaults, and alot of the parameters are minor, so we're mostly going to leave this alone. 

The only thing we'll specify is our output directory, and that we want to see intermediate results every epoch

In [5]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

By default, the model is not evaluated during training (the loss function isn't accuracy, its something like cross-entropy) - we need to be able to pass our Trainer function an evaluation function to have an interpretable way to see what we're doing

In [6]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred # raw outputs, actual labels
    predictions = np.argmax(logits, axis=-1) #prediction is the highest output probability
    return metric.compute(predictions=predictions, references=labels) # accuracy computation

Build the actual Trainer Object

In [7]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

Run it!

This is slow even with a GPU, smaller BERT model, and smaller dataset - but thats the cost of doing buisness with huge models

Bear in mind that with this setup, the model will use your GPU by default if you have one (its using 'mps' on my Apple M1). If you don't have one, this will be even slower.

These models are more often used with external cloud computation/ distributed systems where possible. 

In [8]:
# trainer.train() 

RESULTS:

Time: ~15 minutes on 2022 Mac M1

Epoch 1: 0.493

Epoch 2: 0.579

Epoch 3: 0.588

This hasn't fully converged yet, but it might be starting to slow down.
- Could go farther but your computer might be heating up at this point

Close only counts in horseshoes and handgrenades - we're not rewarding the model for guessing 2/5 when the actual score is 1/5 - thats just as wrong as 5/5

100% accuracy on this task is impossible because not everyone has the same understanding of what each star ratings correspond to - there isn't perfect alignment between the text and the rating

We've also shot ourselves in the foot with a really small training dataset

#### Comparison

In [9]:
small_train_dataset

Dataset({
    features: ['label', 'text', 'input_ids', 'attention_mask'],
    num_rows: 1000
})

In [10]:
type(small_train_dataset['text'])

list

#### Classical ML for Comparison

TF-IDF for representation, Multiclass Naive Bayes for classification

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def preprocess_text(text):
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in ENGLISH_STOP_WORDS])
    return text

X_train = [preprocess_text(text) for text in small_train_dataset['text']]
y_train = small_train_dataset['label']

X_test = [preprocess_text(text) for text in small_eval_dataset['text']]
y_test = small_eval_dataset['label']

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('nb', MultinomialNB())
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.438


We're still beating classical ML/NLP by a fair bit! 

We can imagine that our more complex LLM approach would be able to extract proportionally more value from a larger dataset.

Both would (probably) see increased accuracy on the full dataset, but the LLM moreso.

## Example 2: Summarizing Text

Lets try something we can't do with classical ML!

We're going to summarize text




In [12]:
billsum = load_dataset("billsum", split="ca_test") #legal bill text from California

In [13]:
billsum

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 1237
})

In [14]:
# Shrink it cause the full dataset made my computer memory overflow :(
billsum = billsum.shuffle(seed=42).select(range(100))

In [15]:
billsum

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 100
})

In [16]:
billsum = billsum.train_test_split(test_size=0.2)

#billsum['train'][0]

# We have a long version - our input
# And a summary - our desired output

# Training on the task of transforming a 'text' into its corresponding 'summary', we'll create a legal explanation model

In [17]:
checkpoint = "google-t5/t5-small" # We need an encoder-decoder model since we're going text-text
tokenizer = AutoTokenizer.from_pretrained(checkpoint) # Use the right tokenizer

In [18]:
prefix = "summarize: " # This is a multipurpose model - we need to attach a task to tell it what we want
# Just like how you prompt chat GPT with specific questions


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]] # add summarize to texts
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True) # tokenize inputs
    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True) # tokenize outputs

    model_inputs["labels"] = labels["input_ids"] # match up the text and summary. Specific to this application
    return model_inputs

In [19]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

In [20]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint) # batch examples
# Better for computation and memory
# Also makes it easier/less disruptive to make all sequences in a batch the same length, rather than the whole dataset 

In [21]:
rouge = evaluate.load("rouge") # metric
# specially designed metric for summarization tasks: https://huggingface.co/spaces/evaluate-metric/rouge

In [22]:
# From the documentation, don't worry about syntax here

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}


In [23]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint) # load t5

In [24]:
# Define training args:

training_args = Seq2SeqTrainingArguments(
    output_dir="billsum_model", 
    evaluation_strategy="epoch", 
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01, # regularization in optimizer
    save_total_limit=3, # maximum number of versions to have saved
    num_train_epochs=3,
    predict_with_generate= True
)

In [25]:
# build trainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
) 

In [26]:
import os
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0' # don't try this at home

In [27]:
# trainer.train()
# trainer.save_model('billsum_model')

In [30]:
text = "summarize: The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 35400 of the Vehicle Code is amended to read:\n35400.\n(a) A vehicle may not exceed a length of 40 feet.\n(b) This section does not apply to any of the following:\n(1) A vehicle used in a combination of vehicles when the excess length is caused by auxiliary parts, equipment, or machinery not used as space to carry any part of the load, except that the combination of vehicles shall not exceed the length provided for combination vehicles.\n(2) A vehicle, when the excess length is caused by any parts necessary to comply with the fender and mudguard regulations of this code.\n(3) (A) An articulated bus or articulated trolley coach that does not exceed a length of 60 feet.\n(B) An articulated bus or articulated trolley coach described in subparagraph (A) may be equipped with a folding device attached to the front of the bus or trolley if the device is designed and used exclusively for transporting bicycles. The device, including any bicycles transported thereon, shall be mounted in a manner that does not materially affect efficiency or visibility of vehicle safety equipment, and shall not extend more than 36 inches from the front body of the bus or trolley coach when fully deployed. The handlebars of a bicycle that is transported on a device described in this subparagraph shall not extend more than 42 inches from the front of the bus.\n(4) A semitrailer while being towed by a motortruck or truck tractor, if the distance from the kingpin to the rearmost axle of the semitrailer does not exceed 40 feet for semitrailers having two or more axles, or 38 feet for semitrailers having one axle if the semitrailer does not, exclusive of attachments, extend forward of the rear of the cab of the motortruck or truck tractor.\n(5) A bus or house car when the excess length is caused by the projection of a front safety bumper or a rear safety bumper, or both. The safety bumper shall not cause the length of the vehicle to exceed the maximum legal limit by more than one foot in the front and one foot in the rear. For the purposes of this chapter, “safety bumper” means any device that is fitted on an existing bumper or which replaces the bumper and is constructed, treated, or manufactured to absorb energy upon impact.\n(6) A schoolbus, when the excess length is caused by the projection of a crossing control arm. For the purposes of this chapter, “crossing control arm” means an extendable and retractable device fitted to the front of a schoolbus that is designed to impede movement of pupils exiting the schoolbus directly in front of the schoolbus so that pupils are visible to the driver while they are moving in front of the schoolbus. An operator of a schoolbus shall not extend a crossing control arm while the schoolbus is in motion. Except when activated, a crossing control arm shall not cause the maximum length of the schoolbus to be extended by more than 10 inches, inclusive of any front safety bumper. Use of a crossing control arm by the operator of a schoolbus does not, in and of itself, fulfill his or her responsibility to ensure the safety of students crossing a highway or private road pursuant to Section 22112.\n(7) A bus, when the excess length is caused by a device, located in front of the front axle, for lifting wheelchairs into the bus. That device shall not cause the length of the bus to be extended by more than 18 inches, inclusive of any front safety bumper.\n(8) A bus, when the excess length is caused by a device attached to the rear of the bus designed and used exclusively for the transporting of bicycles. This device may be up to 10 feet in length, if the device, along with any other device permitted pursuant to this section, does not cause the total length of the bus, including any device or load, to exceed 50 feet.\n(9) A bus operated by a public agency or a passenger stage corporation, as defined in Section 226 of the Public Utilities Code, used in transit system service, other than a schoolbus, when the excess length is caused by a folding device attached to the front of the bus which is designed and used exclusively for transporting bicycles. The device, including any bicycles transported thereon, shall be mounted in a manner that does not materially affect efficiency or visibility of vehicle safety equipment, and shall not extend more than 40 inches from the front\nbody\nof the bus when fully deployed. The handlebars of a bicycle that is transported on a device described in this paragraph shall not extend more than 46 inches from the front of the bus. A device described in this paragraph may not be used on a bus that, exclusive of the device, exceeds 40 feet in length or on a bus having a device attached to the rear of the bus pursuant to paragraph (8).\n(10) (A) A bus of a length of up to 45 feet when operating on those highways specified in subdivision (a) of Section 35401.5. The Department of Transportation or local authorities, with respect to highways under their respective jurisdictions, may not deny reasonable access to a bus of a length of up to 45 feet between the highways specified in subdivision (a) of Section 35401.5 and points of loading and unloading for motor carriers of passengers as required by the federal Intermodal Surface Transportation Efficiency Act of 1991 (Public Law 102-240).\n(B) A bus operated by a public agency and on those highways specified in subparagraph (A) may be equipped with a folding device attached to the front of the bus that is designed and used exclusively for transporting bicycles. The device, including all bicycles transported thereon, may be mounted in a manner that does not materially affect efficiency or visibility of vehicle safety equipment, and may not extend more than 36 inches from the front body of the bus when fully deployed. The handlebars of a bicycle that is transported on a device described in this subparagraph may not extend more than 42 inches from the front of the bus. The total length of the bus, including the folding device or load, may not exceed 48.5 feet. A Route Review Committee, established under this subparagraph, shall review the routes where a public agency proposes to operate a 45-foot bus equipped with a front-mounted bicycle rack. The Route Review Committee shall be comprised of one member from the public agency appointed by the general manager of the public agency; one member who is a traffic engineer and is employed and selected by the public agency that has jurisdiction over the largest proportional share of routes among all affected agencies; and one member appointed by the labor organization that is the exclusive representative of the bus drivers of the public agency. If there is no exclusive representative of the bus drivers, a bus driver member shall be chosen by a majority vote of the bus drivers employed by the agency. The members of the Route Review Committee shall be selected not more than 30 days after receipt of a public agency proposal to equip a 45-foot bus with a front-mounted bicycle rack. The review shall include a field review of the proposed routes. The purpose of the Route Review Committee is to ensure the safe operation of a 45-foot bus that is equipped with a front-mounted bicycle rack. The Route Review Committee, by a unanimous vote, shall make a determination of which routes are suitable for the safe operation of a 45-foot bus that is equipped with a front-mounted bicycle rack. These determinations shall be consistent with the operating requirements specified in subparagraph (A). It is the intent of the Legislature that the field review required under this subparagraph include consultation with traffic engineers from affected public agencies that have jurisdiction over segments of the route or routes under review, to ensure coordination with all affected state and local public road agencies that may potentially be impacted due to the operation of a 45-foot bus with a front-mounted bicycle rack.\n(11) (A) A house car of a length of up to 45 feet when operating on the National System of Interstate and Defense Highways or when using those portions of federal aid primary system highways that have been qualified by the United States Secretary of Transportation for that use, or when using routes appropriately identified by the Department of Transportation or local authorities, with respect to highways under their respective jurisdictions.\n(B) A house car described in subparagraph (A) may be operated on a highway that provides reasonable access to facilities for purposes limited to fuel, food, and lodging when that access is consistent with the safe operation of the vehicle and when the facility is within one road mile of identified points of ingress and egress to or from highways specified in subparagraph (A) for use by that vehicle.\n(C) As used in this paragraph and paragraph (10), “reasonable access” means access substantially similar to that authorized for combinations of vehicles pursuant to subdivision (c) of Section 35401.5.\n(D) Any access route established by a local authority pursuant to subdivision (d) of Section 35401.5 is open for access by a house car of a length of up to 45 feet. In addition, local authorities may establish a process whereby access to services by house cars of a length of up to 45 feet may be applied for upon a route not previously established as an access route. The denial of a request for access to services shall be only on the basis of safety and an engineering analysis of the proposed access route. In lieu of processing an access application, local authorities, with respect to highways under their jurisdiction, may provide signing, mapping, or a listing of highways, as necessary, to indicate the use of these specific routes by a house car of a length of up to 45 feet.\n(c) The Legislature, by increasing the maximum permissible kingpin to rearmost axle distance to 40 feet effective January 1, 1987, as provided in paragraph (4) of subdivision (b), does not intend this action to be considered a precedent for any future increases in truck size and length limitations.\n(d) Any transit bus equipped with a folding device installed on or after January 1, 1999, that is permitted under subparagraph (B) of paragraph (3) of subdivision (b) or under paragraph (9) of subdivision (b) shall be additionally equipped with any of the following:\n(1) An indicator light that is visible to the driver and is activated whenever the folding device is in an extended position.\n(2) Any other device or mechanism that provides notice to the driver that the folding device is in an extended position.\n(3) A mechanism that causes the folding device to retract automatically from an extended position.\n(e) (1) A person may not improperly or unsafely mount a bicycle on a device described in subparagraph (B) of paragraph (3) of subdivision (b), or in paragraph (9) or (10) of subdivision (b).\n(2) Notwithstanding subdivision (a) of Section 23114 or subdivision (a) of Section 24002 or any other provision of law, when a bicycle is improperly or unsafely loaded by a passenger onto a transit bus, the passenger, and not the driver, is liable for any violation of this code that is attributable to the improper or unlawful loading of the bicycle.\nSECTION 1.\nSection 37420.5 is added to the\nGovernment Code\n, to read:\n37420.5.\n(a)The City of Montebello may sell all or part of its water utility pursuant to this article as provided in Section 10051.5 of the Public Utilities Code.\n(b)This section shall remain in effect only until July 1, 2017, and as of that date is repealed, unless a later enacted statute, that is enacted before July 1, 2017, deletes or extends that date.\nSEC. 2.\nSection 10051.5 is added to the\nPublic Utilities Code\n, to read:\n10051.5.\n(a)Notwithstanding any other provision of this article, the City of Montebello may sell all or part of its water utility property pursuant to Article 3 (commencing with Section 37420) of Chapter 5 of Part 2 of Division 3 of Title 4 of the Government Code, subject and pursuant to the following additional provisions, limitations, and requirements:\n(1)The governing body of the city shall not sell the water utility property for less than its fair market value determined as set forth in the Public Water System Investment and Consolidation Act of 1997 (Chapter 2.5 (commencing with Section 2718) of Part 2 of Division 1).\n(2)A majority of the members of the governing body of the city must approve the sale.\n(3)The entity acquiring the water utility property shall be a public utility, as defined in Section 216, which is presently authorized by the commission to provide water utility service within the corporate limits of the city pursuant to a certificate of public convenience and necessity and the area of the one or more certificates includes or surrounds all or part of the city’s water utility system.\n(4)As a condition of acquiring the city’s water system, the acquiring public utility shall file a revised service area map with the commission that shows the area formerly served through the city’s water utility system as part of the acquiring public utility’s service area. The acquiring public utility shall submit the map by filing a tier-one advice letter with the commission and, upon this filing, shall be deemed authorized and bound to render public utility water service to all persons in the area formerly served through the city’s water utility system on the same terms, conditions, and rates that the commission has found to be just and reasonable for the acquiring public utility’s other customers in the city.\n(b)This section shall remain in effect only until July 1, 2017, and as of that date is repealed, unless a later enacted statute, that is enacted before July 1, 2017, deletes or extends that date.\nSEC. 3.\nThe Legislature finds and declares that a special law is necessary and that a general law cannot be made applicable within the meaning of Section 16 of Article IV of the California Constitution because of the unique circumstances applicable to the City of Montebello. The City Council of the City of Montebello believes it is no longer in the public interest to own and operate all or part of a water utility and desires to sell its water utility. However, the city would be unable to do so in a timely manner because the Legislature has enacted statutes that may conflict with one another and with the City of Montebello’s urgent financial requirements, thereby preventing the City of Montebello from selling and the acquiring public utility from purchasing the water utility without an election and performance of other procedures. In order to clearly state the law with respect to the City of Montebello, a special statute is needed and a general statute cannot be made applicable.\nSEC. 4.\nThis act is an urgency statute necessary for the immediate preservation of the public peace, health, or safety within the meaning of Article IV of the Constitution and shall go into immediate effect. The facts constituting the necessity are:\nThe City of Montebello’s water utility infrastructure is aging and is experiencing numerous leaks and equipment failures. The aging and failing water utility infrastructure impairs the safety and reliability of the city’s water utility system, including its ability to provide water for public fire-fighting purposes. The city’s water utility serves only a small fraction of the city’s residents and is losing money despite significant recent rate increases. The city’s budget and general fund cannot make up for that revenue shortfall or the cost of the urgently needed water supply infrastructure upgrades and repairs of its water utility system. The City of Montebello desires to sell all or part of the city-owned water utility to a public utility capable of operating, managing, and upgrading the water system while financial conditions are favorable to the city, however, the Legislature has adopted statutes governing the sale of municipal property which may conflict with each other and the urgent needs of the city. In order to provide the City of Montebello with the clear statutory authority to sell its water utility without an election as soon as possible, it is necessary that this act take effect immediately."
 

This is ~2720 words

In [33]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("billsum_model", local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained("billsum_model", local_files_only=True)

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)

# Generate summary
summary_ids = model.generate(inputs["input_ids"], num_beams=4, min_length=None, max_length= 500, early_stopping=True)

# Decode the summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print the summary
print("Summary:", summary)


Summary: an articulated bus or articulated trolley coach that does not exceed a length of 60 feet. an articulated bus or articulated trolley coach may be equipped with a folding device attached to the front of the bus or trolley if the device is designed and used exclusively for transporting bicycles. the device, including any bicycles transported thereon, shall not extend more than 42 inches from the front body of the bus.
